In [ ]:
%run preprocecssing_helpers.ipynb
%run Batch_ArtifactFilter_Epoch.ipynb
import os
import time
import mne
from mne.preprocessing import ICA
import numpy as np
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 20.0)
matplotlib.rcParams.update({'font.size': 15})
# matplotlib.rcParams.

In [ ]:
# path = 'F:\StroopEEG'
path = 'E:\EEGLAB'

eeglab_files = os.listdir(path)
eeglab_dict = search_folders(path, eeglab_files)
eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
eeglab_files;

In [ ]:
raw = mne.io.read_raw_eeglab(eeglab_files[3], preload = True)

In [ ]:
raw.get_data().shape

In [ ]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [ ]:
def create_epoch_data_array(raw_data, events):
    #get 500ms after each stim-code
    epochArray = []
    for event in events:
        if event[-1] <=24:
            start_sample = event[0]
            epochArray.append(raw_data[:,start_sample:start_sample + round(.5 *512)])
    return np.array(epochArray)

In [ ]:
def events_with_stim_codes(events):
    """
    Description:
        Method will return an mne events ndarray containing only 
        stim-codes (event_ids 1-24).
    Variables:
        events: list of 3d lists 
    -----
    Returns:
        new_events list of 3d lists containing events of stim-codes    
    """
    new_events = []
    for event in events:
        if event[-1] <= 24:
            new_events.append(np.array(event))
            
    return np.array(new_events)

In [ ]:
def create_Epoch_Object(raw, data, events):
    # create info
    ch_names = raw.ch_names
    sfreq = raw.info['sfreq']
    montage = mne.channels.read_montage('ANT_DukeWaveGuard_128_electrode_montages_updated_V4')
    
    info = mne.create_info(ch_names=ch_names, ch_types= ['eeg']*129, sfreq= sfreq, montage= montage)
#     raw_events = mne.find_events(raw, output= 'offset',
# #                               consecutive= True,
#                               min_duration=sfreq, 
#                               shortest_event= .5, 
#                               initial_event=False)
#     events = events_with_stim_codes(raw_events)
    print(events.shape)
    event_id = new_stim_code_event_ids;
    print(raw.get_data().shape)
#     data = create_epoch_data_array(raw.get_data(), events =events)
    print(data.shape)
    return mne.EpochsArray(data, info, events, 0, event_id)

In [ ]:
samples = raw.get_data().shape[-1]
seconds = 1672.503906
print("Samples: {0}".format(samples))
print("Seconds: {0}".format(seconds))

In [ ]:
raw_events = mne.find_events(raw, output= 'offset',
#                               consecutive= True,
                              min_duration=1/raw.info['sfreq'], 
                              shortest_event= .5, 
                              initial_event=False)
events = events_with_stim_codes(raw_events)

In [ ]:
ica_data = artifact_removal(raw)

In [ ]:
ica_data.get_data().shape

In [ ]:
epochArray = create_epoch_data_array(ica_data.get_data(), events=events)

In [ ]:
epochArray.shape

In [ ]:
raw_data = raw.get_data()[:,:100]

In [ ]:
raw_data.shape

In [ ]:
raw_events.shape

In [ ]:
epoch = create_Epoch_Object(raw,epochArray, events)

In [ ]:
# epoch.get_data().shape
# resampled_epoch = epoch.resample(sfreq = 256)

In [ ]:
epoch.plot(title="epochs", n_channels=20, );

In [ ]:
epoch.plot(title="epochs", n_channels=20, );

In [ ]:
# montage = mne.channels.read_montage('ANT_DukeWaveGuard_128_electrode_montages_updated_V4')
# epoch.set_montage(montage = montage)

In [ ]:
evoked = epoch.average();

In [ ]:
vmin = evoked.data.min()
vmax = evoked.data.max()

In [ ]:
plt.plot(epoch.get_data()[1][:128][10]);

In [ ]:
plt.boxplot(evoked.data)
# plt.yticks(np.arange(-.5,2,.1))

In [ ]:
evoked.plot_topomap(time_unit='ms', ch_type='eeg', vmin = -.6, vmax = .6 );

In [ ]:
epoch.plot_psd_topomap( );

In [ ]:
np.max(epoch.get_data())

In [ ]:
create_epoch_data_array(raw.get_data(), events).shape

In [ ]:
raw

In [ ]:
raw

In [ ]:
events = mne.find_events(raw, output= 'offset',
#                               consecutive= True,
                              min_duration=1/raw.info['sfreq'], 
                              shortest_event= 0, 
                              initial_event=False)

In [ ]:
events.shape

In [ ]:
epoch = mne.read_epochs('20140205_1114_epo.fif')

In [ ]:
epoch.events

In [ ]:
epoch.plot_sensors();

In [ ]:
if 'Nasium' in epoch.ch_names:
    epoch.drop_channels(ch_names=['Nasium', 'LL4', 'L12', 'VEOG']);
# epoch.plot_psd_topomap();

In [ ]:
evoked = epoch.average()
evoked.plot_topomap(times=[.01,.50,.100,.150,.250,.300,.350,.400,.450,.500]);

In [ ]:
3931.406250 *512

In [ ]:
epoch.get_data().shape

In [ ]:
epoch.events.shape